In [1]:
from ngboost import NGBRegressor
from scipy.stats import expon
from scipy.stats import multivariate_normal
from sklearn.datasets import load_boston
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from ngboost.scores import CRPScore, LogScore
from ngboost.distns import Exponential
from ngboost.distns import MultivariateNormal
from pandas import read_csv
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd
import pickle

def backup(object, filename):
    with open(filename, 'wb') as f:
        pickle.dump(object, f, pickle.HIGHEST_PROTOCOL)

def reload(persist):
    with open(persist, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        data = pickle.load(f)
    return data

# series = read_csv('ProcessedDataCenterHall.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
#series = read_csv('ProcessedBioEngineeringPV.csv', header=0, parse_dates=True)
outputs_feature_select = reload("outputs_feature_select.pickle")



In [2]:
dataset = pd.read_csv("supervised_dataset.csv.gz")

In [3]:
dataset.set_index("DateTime",inplace=True)

In [4]:
dataset = dataset.iloc[:2000,:]

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset.iloc[:,:24*4], dataset.iloc[:,-24:], test_size=0.2, shuffle=False)
# X_train, X_test, Y_train, Y_test = train_test_split(x_series_value, y_date_numerical, test_size=0.2)

In [6]:

ngb = NGBRegressor(Dist=MultivariateNormal(k=24), Score=LogScore, minibatch_frac=0.05, verbose=True).fit(X_train, Y_train)

[iter 0] loss=65.4954 val_loss=0.0000 scale=0.0312 norm=4.6848
[iter 100] loss=63.9191 val_loss=0.0000 scale=0.0312 norm=4.6865
[iter 200] loss=65.5286 val_loss=0.0000 scale=0.0312 norm=4.9900
[iter 300] loss=63.3174 val_loss=0.0000 scale=0.0312 norm=4.4799
[iter 400] loss=63.1620 val_loss=0.0000 scale=0.0312 norm=4.1245


In [7]:
Y_preds = ngb.predict(X_test)
Y_dists = ngb.pred_dist(X_test)

# Deterministic Values
print("Y_preds: ", Y_preds)
print("Y_dist: ", Y_dists)

# Probabilistic Values
mean = Y_dists.mean
sample = Y_dists.rv()
scipy_list = Y_dists.scipy_distribution()

print("mean: ", mean)
print("sample: ", sample)

# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, Y_test)
print('Test MSE', test_MSE)

# test Negative Log Likelihood
test_NLL = -Y_dists.logpdf(Y_test).mean()
print('Test NLL', test_NLL)


Y_preds:  [[107.79837341  48.0059034  107.62311146 ...  47.29899154 106.77309607
   47.29183322]
 [107.7465496   47.90601593 107.57701689 ...  47.29451427 106.79964305
   47.3244107 ]
 [107.681228    47.84629273 107.53260012 ...  47.29012309 106.79736281
   47.2986065 ]
 ...
 [106.90176213  46.91956051 106.82911704 ...  46.76741107 106.71652819
   46.89655133]
 [106.88769726  46.90462488 106.83973922 ...  47.15628697 107.70715935
   47.59002431]
 [106.89422059  46.89938236 106.84520617 ...  48.10511636 112.11948831
   50.08363837]]
Y_dist:  <ngboost.distns.multivariate_normal.MultivariateNormal.<locals>.MVN object at 0x7f9c90074cd0>
mean:  <bound method MultivariateNormal.<locals>.MVN.mean of <ngboost.distns.multivariate_normal.MultivariateNormal.<locals>.MVN object at 0x7f9c90074cd0>>
sample:  [[ 65.67090035  35.52266026  57.76417358 ...  28.74556238  59.90803881
   35.46858385]
 [ 63.58574661  30.44104387  57.24483064 ...  41.0631515   79.94143525
   40.8938481 ]
 [131.95505226  60.4

In [8]:
dataset

,realpowerin_0,reactivepowerin_0,realpowerin_1,reactivepowerin_1,realpowerin_2,reactivepowerin_2,realpowerin_3,reactivepowerin_3,realpowerin_4,reactivepowerin_4,...,realpower_91,reactivepower_91,realpower_92,reactivepower_92,realpower_93,reactivepower_93,realpower_94,reactivepower_94,realpower_95,reactivepower_95
DateTime,,,,,,,,,,,,,,,,,,,,,
2016-01-21 10:30:00,167.978,67.786,172.388,69.468,185.135,64.722,181.840,66.309,183.121,71.613,...,168.736,73.000,168.875,74.799,177.513,77.711,186.928,74.418,185.142,73.786
2016-01-21 10:45:00,169.429,68.395,167.978,67.786,172.388,69.468,185.135,64.722,181.840,66.309,...,168.875,74.799,177.513,77.711,186.928,74.418,185.142,73.786,184.099,72.812
2016-01-21 11:00:00,187.743,69.617,169.429,68.395,167.978,67.786,172.388,69.468,185.135,64.722,...,177.513,77.711,186.928,74.418,185.142,73.786,184.099,72.812,190.646,78.394
2016-01-21 11:15:00,183.077,67.184,187.743,69.617,169.429,68.395,167.978,67.786,172.388,69.468,...,186.928,74.418,185.142,73.786,184.099,72.812,190.646,78.394,189.016,79.262
2016-01-21 11:30:00,181.433,65.150,183.077,67.184,187.743,69.617,169.429,68.395,167.978,67.786,...,185.142,73.786,184.099,72.812,190.646,78.394,189.016,79.262,184.372,76.516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-02-11 05:15:00,62.121,25.460,61.473,25.067,61.001,24.555,61.206,24.493,61.840,24.618,...,62.690,26.442,63.189,27.143,63.982,27.434,63.075,26.623,63.965,26.454
2016-02-11 05:30:00,61.322,25.076,62.121,25.460,61.473,25.067,61.001,24.555,61.206,24.493,...,63.189,27.143,63.982,27.434,63.075,26.623,63.965,26.454,64.357,27.445
2016-02-11 05:45:00,65.893,28.540,61.322,25.076,62.121,25.460,61.473,25.067,61.001,24.555,...,63.982,27.434,63.075,26.623,63.965,26.454,64.357,27.445,66.777,30.199
